<a href="https://www.kaggle.com/code/zoendeloi/proyecto-gdd-sofiafraile-fabiangil-enzoloiza?scriptVersionId=141690294" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

[Open in Kaggle](https://www.kaggle.com/code/zoendeloi/proyecto-gdd-sofiafraile-fabiangil-enzoloiza)

# Proyecto Grupal

### Gestión de Datos

Sofía Fraile, Fabián Gil y Enzo Loiza

El siguiente documento corresponde a un informe descriptivo de información rescatada de tres bases de datos: (1) Asistencias a colegios de todo Chile (MINEDUC), (2) ubicación geográfica de los colegios (Propia/Google Maps), y (3) información meteorológica de calidad ambiental (SINCA). Contiene tres partes.

En la primera parte, se realizó un análisis descriptivo de las variables que contienen las bases de datos, y se trabajaron tales variables para que compartan formato entre ellas para simplificar su lectura. En la segunda, se describe el proceso de poblar las bases de datos en SQL. Finalmente, en la tercera parte, se respondieron algunas preguntas relacionadas con las bases de datos.

In [1]:
gc()
rm(list=ls())

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,959928,51.3,1949610,104.2,1249132,66.8
Vcells,1702482,13.0,8388608,64.0,2651364,20.3


In [2]:
# libraries
library("tidyverse")
library("data.table")
library("dplyr")
library("ggplot2")
library("DBI")
library("RPostgreSQL")
library("RODBC")
library("RPostgres")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




## Documentación de la Base de Datos

Las bases de datos a cargar fueron tres: Asistencias colegios 2018-2019, Calidad de Aire Chile y Localizacion geográfica de colegios, cada una con las variables que se mencionan a continuación:

**Asistencias colegios 2018-2019 (26 variables)**

| N | Nombre | Tipo | Descripción|
|---|-----|------|------------|
| 1 | AGNO | Numeric | Año del proceso |
| 2 | MES_ESCOLAR | Numeric | Mes |
| 3 | RBD | Numeric | Rol Base de Datos del Establecimiento |
| 4 | DGV_RBD | Numeric | Dígito verificador del RBD |
| 5 | NOM_RBD | Character | Nombre del establecimiento |
| 6 | COD_REG_RBD | Numeric | Codigo de region en que se ubica el establecimiento |
| 7 | COD_REG_RBD_A | Character | Nombre abreviado de la región en que se ubica el establecimiento |
| 8 | COD_PRO_RBD | Numeric | Codigo de la provincia en que se ubica el establecimiento |
| 9 | COD_COM_RBD | Numeric | Código de oficial de comuna en que se ubica el establecimiento |
| 10 | NOM_COM_RBD | Character | Nombre de la comuna |
| 11 | COD_DEPROV_RBD | Numeric | código del departamento provincial en que se ubica el establecimiento |
| 12 | NOM_DEPROV_RBD | Character | Nombre del departamento provinicial donde se ubica el establecimiento | 
| 13 | RURAL_RBD | Numeric | 0 si urbano, 1 si rural |
| 14 | COD_DEPE | Numeric | Dependencia administrativa |
| 15 | COD_DEPE2 | Numeric | Código de dependencia administrativa (agrupado) |
| 16 | COD_ENSE | Numeric | Tipo de enseñanza |
| 17 | COD_ENSE2 | Numeric | Niveles de enseñanza agrupados | 
| 18 | COD_GRADO | Numeric | Código de grado | 
| 19 | LET_CUR | Character | Letra del curso | 
| 20 | MRUN | Numeric | Identificador estudiante máscara | 
| 21 | GEN_ALU | Numeric | Sexo del estudiante |
| 22 | FEC_NAC_ALU | Numeric | Fecha nacimiento alumno AAAAMM (190001/180001 sin info) |
| 23 | COD_COM_ALU | Numeric | Codigo oficial de la comuna de residencia del alumno |
| 24 | NOM_COM_ALU | Character | Nombre comuna de residencia |
| 25 | DIAS_ASISTIDOS | Numeric | Total numero de días asistidos |
| 26 | DIAS_TRABAJADOS | Numeric | Total numero de días trabajados |
| 27 | ASIS_PROMEDIO | Numeric | proporcion de días asistidos respecto de días trabajados |

**Calidad de Aire Chile (12 variables)**

| N | Variable | Tipo | Descripción |
|---|----------|------|-------------|
| 1 | Estacion | Chr | Nombre de la comuna donde está la estación |
| 2 | Fecha (YYMMDD) | int | Fecha de medicion en formato YYMMDD |
| 3 | HORA (HHMM) | int | Hora de medicion en formato HHMM |
| 4 | MP10 | Chr | Material particulado de 10 micrones (decimales con coma) |
| 5 | MP2_5 | Chr | Material part 2.5 micrones (decimales con coma) | 
| 6 | CO | Chr | Monóxido de carbono |
| 7 | NO | Chr | Monoxido nitroso | 
| 8 | NO2 | Chr | Dioxido nitroso |
| 9 | O3 | Chr | Ozono |
| 10 | Humedad | Chr | idem |
| 11 | Temperatura | Chr | idem |
| 12 | Viento_v | Chr | viento |

Localizacion geográfica de colegios (6 variables)

| N | Variable | Tipo | Descripción |
|---|----------|------|-------------|
| 1 | RBD | int | Rol de base de datos del colegio |
| 2 | NOM_RBD | chr | Nombre colegio |
| 3 | NOM_COM_RBD | chr | Nombre comuna del establecimiento |
| 4 | query | chr | union de 2 y 3 para busqueda en maps | 
| 5 | lon | chr | longitud geográfica |
| 6 | lat | chr | latitud geográfica |

Sin embargo, las variables no estaban todas en un formato común, por lo que se debió trabajar en ellas para poder consolidarlas.

La primera base de datos (Asistencias colegios 2018-2019) provenía de 20 archivos `csv` donde algunas de éstas tenían 27 variables y otras 26. La variable diferente correspondió a `NOM_REG_RBD_A`, y para efectos de este informe se decidió eliminar la columna. La base se guardó como `db`.

In [3]:
nombre_archivos1 <- c("/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180816_Asistencia_Marzo_2018_20180415_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180830_Asistencia_Abril_2018_20180515_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180720_Asistencia_Mayo_2018_20180615_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180816_Asistencia_Junio_2018_20180715_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180904_Asistencia_Julio_2018_20180815_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20181026_Asistencia_Agosto_2018_20180915_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20181127_Asistencia_Septiembre_2018_20181015_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20181213_Asistencia_Octubre_2018_20181115_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20190107_Asistencia_Noviembre_2018_20181215_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20190121_Asistencia_Diciembre_2018_20190115_PUBL.csv")
nombre_archivos2 <- c("/kaggle/input/asistencia-colegios-2018-y-2019-chile/20190514_Asistencia_Marzo_2019_20190415_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20190812_Asistencia_Abril_2019_20190515_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20191511_Asistencia_Mayo_2019_20190615_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20191211_Asistencia_Junio_2019_20190715_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20191211_Asistencia_Julio_2019_20190715_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20191227_Asistencia_Agosto_2019_20190915_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20200102_Asistencia_Septiembre_2019_20191015_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20200106_Asistencia_Octubre_2019_20191115_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20200108_Asistencia_noviembre_2019_20191215_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20200120_Asistencia_diciembre_2019_20200115_PUBL.csv")

db1 <- data.table()
db2 <- data.table()

columna_a_eliminar <- "NOM_REG_RBD_A"

In [4]:
for (archivo in nombre_archivos1) {
    data_temp <- fread(archivo, "encoding" ="UTF-8")
    if (columna_a_eliminar %in% colnames(data_temp)){
        data_temp[, (columna_a_eliminar) := NULL]
    }
    db1 <- rbindlist(list(db1, data_temp))
}

In [5]:
for (archivo in nombre_archivos2) {
    data_temp <- fread(archivo, "encoding" ="UTF-8")
    if (columna_a_eliminar %in% colnames(data_temp)){
        data_temp[, (columna_a_eliminar) := NULL]
    }
    db2 <- rbindlist(list(db2, data_temp))
}

Por el tiempo que toma modificar esta tabla (de alrededor de 7 millones de instancias), se decidió que las variables de las otras bases de datos sean modificadas para parecerse a ésta. La segunda parte de la base de datos se guardó como `db2` y correspondió a los índices de calidad de aire. En esta parte se trabajaron las variables como sigue.

In [6]:
db3 <- fread("/kaggle/input/calidad-del-aire-en-chile/Calidad del aire.csv", "encoding" ="UTF-8")

In [7]:
# ajustes de fechas a formato de db
db3$Fecha <- as.character(db3$"FECHA (YYMMDD)")
db3 <- db3 %>%
    mutate(
        AGNO = as.integer(substr(Fecha, 1, 2)) +
        if_else(as.integer(substr(Fecha, 1, 2)) >= 50, 1900, 2000),
        MES = as.integer(substr(Fecha, 3, 4)),
        DIA = as.integer(substr(Fecha, 5, 6))
    ) %>%
    select(-Fecha) %>%
    select(-"FECHA (YYMMDD)")

# ajustes a MP10, MP2_5, CO, NO, NO2, O3, Humedad, Temperatura, Viento_v
db3 <- db3 %>%
    mutate(
        MP10 = as.numeric(gsub(",", ".", MP10)),
        MP2_5 = as.numeric(gsub(",", ".", MP2_5)),
        CO = as.numeric(gsub(",", ".", CO)),
        NO = as.numeric(gsub(",", ".", NO)),
        NO2 = as.numeric(gsub(",", ".", NO2)),
        O3 = as.numeric(gsub(",", ".", O3)),
        Humedad = as.numeric(gsub(",", ".", Humedad)),
        Temperatura = as.numeric(gsub(",", ".", Temperatura)),
        Viento_v = as.numeric(gsub(",", ".", Viento_v))
    )

In [8]:
str(db3)
summary(db3$MP2_5)

Classes ‘data.table’ and 'data.frame':	47953 obs. of  14 variables:
 $ Estacion   : chr  "Pudahuel" "Pudahuel" "Pudahuel" "Pudahuel" ...
 $ HORA (HHMM): int  0 0 0 0 0 0 0 0 0 0 ...
 $ MP10       : num  115 102 647 799 843 ...
 $ MP2_5      : num  NA NA NA NA NA NA NA NA NA NA ...
 $ CO         : num  NA NA NA NA NA ...
 $ NO         : num  NA NA NA NA NA NA NA NA NA NA ...
 $ NO2        : num  NA NA NA NA NA NA NA NA NA NA ...
 $ O3         : num  354.2 385.8 33.2 400.8 325.8 ...
 $ Humedad    : num  NA NA NA NA NA NA NA NA NA NA ...
 $ Temperatura: num  NA NA NA NA NA NA NA NA NA NA ...
 $ Viento_v   : num  NA NA NA NA NA NA NA NA NA NA ...
 $ AGNO       : num  1997 1997 1997 1997 1997 ...
 $ MES        : int  4 4 4 4 4 4 4 4 4 4 ...
 $ DIA        : int  4 5 6 7 8 9 10 11 12 13 ...
 - attr(*, ".internal.selfref")=<externalptr> 


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  0.708  21.375  93.125 171.272 254.167 998.333    9922 

Además, incluimos en esta parte las coordenadas de las estaciones de monitoreo.

In [9]:
unique(db3$Estacion) #'Pudahuel''Las Condes''Santiago''El Bosque''La Florida'



[1] "Pudahuel"   "Las Condes" "Santiago"   "El Bosque"  "La Florida"

## Poblar Bases de Datos

Lorem ipsum.

## Preguntas Específicas

Lorem ipsum.

In [10]:
str(db1)

Classes ‘data.table’ and 'data.frame':	34910120 obs. of  26 variables:
 $ AGNO           : int  2018 2018 2018 2018 2018 2018 2018 2018 2018 2018 ...
 $ MES_ESCOLAR    : int  3 3 3 3 3 3 3 3 3 3 ...
 $ RBD            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ DGV_RBD        : int  9 9 9 9 9 9 9 9 9 9 ...
 $ NOM_RBD        : chr  "LICEO POLITECNICO ARICA" "LICEO POLITECNICO ARICA" "LICEO POLITECNICO ARICA" "LICEO POLITECNICO ARICA" ...
 $ COD_REG_RBD    : int  15 15 15 15 15 15 15 15 15 15 ...
 $ COD_PRO_RBD    : int  151 151 151 151 151 151 151 151 151 151 ...
 $ COD_COM_RBD    : int  15101 15101 15101 15101 15101 15101 15101 15101 15101 15101 ...
 $ NOM_COM_RBD    : chr  "ARICA" "ARICA" "ARICA" "ARICA" ...
 $ COD_DEPROV_RBD : int  151 151 151 151 151 151 151 151 151 151 ...
 $ NOM_DEPROV_RBD : chr  "ARICA" "ARICA" "ARICA" "ARICA" ...
 $ RURAL_RBD      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ COD_DEPE       : int  2 2 2 2 2 2 2 2 2 2 ...
 $ COD_DEPE2      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ COD_ENSE    